In [ ]:
# ----- IMPORTACIÓN DE LIBRERÍAS -----
# Se importan las librerias necesarias para proceder con la obtención de datos de Google Trends

import pandas as pd
from pytrends.request import TrendReq
import time

print("Librerías importadas correctamente.")

# Importación de Librerias
Esta primera celda, nos permite cargar las librerias fundamentales para el proyecto que estamos realizando
'pandas:' Esta es la libreria estandar en Python para la manipulación y análisis de los datos (DataFrames)
TrendReq de pytrends: Nos permite conectar a la API de Google Trends para realizar nuestra consultas de las tendencias 
time: LIbreria estandar de Python que nos permite agregar pausas deliberadas entre peticiones

In [ ]:
# ----- DEFINICIÓN DE PARAMETROS DE BUSQUEDA -----
# Se definen los parametros de busqueda que se utilizaran en las consultas a Google Trends
# Centralizamos todas las variables de la consulta

# Lista de términos a buscar. Pytrends funciona mejor con listas de 5 términos
lista_keywords = ["laptop", "iphone", "samsung galaxy", "playstation 5", "tarjeta grafica"]

# Parametros geográficos y temporales  codigo ISO 3166 alpha-2 para Ecuador es "EC"
pais ="EC"

#Periodo de tiempo para que sea desde el 1 de enero del 2024 hasta la fecha actual
periodo_tiempo = '2024-01-01 today'

print(f"Parámetros definidos. Se analizarán los terminos: {lista_keywords}")
print(f"La búsqueda se realizará en '{pais}' desde '{periodo_tiempo}'.")

lista_keywords

### DEFINICIÓN DE PARAMETROS
Aqui definimos las variables que controlarán nuestra extracción de datos. Si quisieramos analizar otros productos, un pais diferente o un rango diferente de fechas, solo necesitamos modificar esta celda.
pais: Se especifica 'EC' resultados exclusivos a Ecuador.
periodo_tiempo: se define el rango de la extraccion de la información desde el primero de enero del 2024 hasta la actualidad

In [ ]:
### -----EXTRACCIÓN DE DATOS -----
# Conexion a la API y se procede a realizar las dos consultas principales
# 1. Tendencias a lo largo del tiempo
# 2. Tendencias por subregión

# Inicializa la conexión  con Goole Trends especificando el idioma y el uso horarios (UTC-5 --> 300 minutos)
# Conexion a traves de un proxy para evitar bloqueos por parte de Google
"""
pytrends = TrendReq(hl='es-EC', tz=300, requests_args={'proxies': {'https', 'https://proxy.google.com'}})

# 1. Tendencias a lo largo del tiempo
print("Iniciando extracción de datos de tendencias a lo largo del tiempo...")
try:
    pytrends.build_payload(kw_list=lista_keywords, timeframe=periodo_tiempo, geo=pais)
    df_tendencias = pytrends.interest_over_time()

    print("Extracción de datos de tendencias a lo largo del tiempo completada.")
except Exception as e:
    print(f"Error durante la extracción de datos de tendencias a lo largo del tiempo: {e}")
    # Crear un DataFrame vacío 
    df_tendencias = pd.DataFrame()

# 2. Tendencias por subregión
print("\nIniciando extracción de datos de tendencias por subregión...")

# Pausas para evitar saturar el servicio
time.sleep(10)
try:
    df_regiones = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True)
    print("\n¡Consulta de datos completada!")
except Exception as e:
    print(f"Error durante la extracción de datos de tendencias por subregión: {e}")
    # Crear un DataFrame vacío 
    df_regiones = pd.DataFrame()

print("\n¡EXTRACCION DE DATOS COMPLETADA!")
"""
# Definimos los argumentos para la librería requests
requests_args = {
    'proxies': {
        'https': 'https://proxy.google.com'
    },
    'timeout': (10, 30) # 10s para conectar, 30s para leer
}

# Inicializamos la conexión con todas las opciones:
# hl: idioma, tz: zona horaria, timeout, reintentos, y los argumentos de requests (que incluyen el proxy).
try:
    print("Estableciendo conexión con Google Trends...")
    pytrends = TrendReq(
        hl='es-EC', 
        tz=300, 
        timeout=(10, 30), 
        retries=2, 
        backoff_factor=0.1, 
        requests_args=requests_args
    )
    print("Conexión establecida.")
except Exception as e:
    print(f"Error al inicializar la conexión: {e}")
    pytrends = None

# Solo proceder si la conexión fue exitosa
if pytrends:
    # 1. Obtenemos el interés a lo largo del tiempo
    print("\nRealizando la primera consulta: Interés a lo largo del tiempo...")
    try:
        pytrends.build_payload(kw_list=lista_keywords, timeframe=periodo_tiempo, geo=pais)
        df_tendencias = pytrends.interest_over_time()
        print("-> Consulta de tendencias temporales exitosa.")
    except Exception as e:
        print(f"-> ERROR al obtener tendencias temporales: {e}")
        df_tendencias = pd.DataFrame() # Crear un DataFrame vacío si falla

    # 2. Obtenemos el interés por región
    print("\nRealizando la segunda consulta: Interés por región...")
    time.sleep(10)  # Pausa de 10 segundos para ser respetuosos con la API

    try:
        # No es necesario volver a llamar a build_payload si los keywords no han cambiado
        df_regiones = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True)
        print("-> Consulta de interés por región exitosa.")
    except Exception as e:
        print(f"-> ERROR al obtener interés por región: {e}")
        df_regiones = pd.DataFrame() # Crear un DataFrame vacío si falla

    print("\n¡Extracción de datos finalizada!")
else:
    print("No se pudo continuar con la extracción de datos debido a un error de conexión.")
    # Creamos DataFrames vacíos para que el resto del notebook no falle
    df_tendencias = pd.DataFrame()
    df_regiones = pd.DataFrame()


###     ----- IMPORTAR LIBRERIAS DE DE VIZUALIZACIÓN -----
En esta fase, importamos matplotlib y seaborn, las librerías estándar para la visualización de datos en Python. Se establece un estilo visual (whitegrid) y un tamaño de figura por defecto para asegurar que nuestros gráficos sean legibles y estéticamente atractivos.

In [ ]:
# importar librerias de vizualización
import matplotlib.pyplot as plt
import seaborn as sns

#configuración de estilo para los graficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (15, 7)  # Tamaño por defecto de las figuras

print("Librerías de visualización importadas y configuradas.")



### ----- ¿CÚAL ES EL PRDUCTO MAS POPULAR? -----

Para responder a nuestra primera pregunta, calculamos la media del interés de búsqueda para cada producto a lo largo del tiempo. El método .mean() de Pandas calcula el promedio de cada columna, y .sort_values(ascending=False) nos permite ordenarlos de mayor a menor.
Insight: El gráfico de barras muestra claramente el ranking de popularidad. [Aquí Comp3 debe escribir una breve conclusión, ej: "El término 'laptop' y 'playstation 5' dominan el interés de búsqueda promedio, sugiriendo una demanda constante y alta para estos productos..."]

In [ ]:
# ----- GRAFICO DE TENDENCIAS TEMPORALES -----
# Visualizamos cómo ha cambiado el interés de cada producto a lo largo del tiempo.

df_tendencias.plot(kind='line', style='-o', ms=4)
plt.title('Evolución del Interés de Búsqueda a lo largo del Tiempo (2024)', fontsize=16)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Interés de Búsqueda Normalizado (0-100)', fontsize=12)
plt.legend(title='Productos')
plt.show()

# ----- ¿Cómo ha evolucionado el interés? -----

El gráfico de líneas nos permite visualizar la dinámica de la demanda. Cada línea representa un producto, mostrando sus picos y valles a lo largo de las semanas. El valor 100 en el eje Y no representa un número absoluto de búsquedas, sino el punto de máxima popularidad para el término más buscado en el período y región seleccionados; todos los demás valores son relativos a ese pico.



In [ ]:
# --- Celda 17: Análisis Geográfico ---
# Analizamos qué producto es el más popular en cada provincia.

# Creamos una nueva columna con el producto más popular por región
df_regiones['producto_top'] = df_regiones.idxmax(axis=1)

print("--- Producto más popular por Provincia ---")
print(df_regiones['producto_top'])

# Visualizamos la distribución
plt.figure(figsize=(12, 6))
sns.countplot(y=df_regiones['producto_top'], order=df_regiones['producto_top'].value_counts().index, palette='plasma')
plt.title('Distribución del Producto Más Popular por Provincia', fontsize=16)
plt.xlabel('Número de Provincias donde es #1', fontsize=12)
plt.ylabel('Producto Tecnológico', fontsize=12)
plt.show()